###  output code

In [24]:
import numpy as np
import pandas as pd
import json

In [71]:
predict = pd.read_csv('transform_res_final.csv')

In [72]:
# str으로 들어가있어서 float로 바꾸는 과정입니다.
for i in range(2000):
    predict['종가'][i] = [float(item) for item in predict['종가'][i].strip('[]').split(',')]

C:\Users\admin\AppData\Local\Temp\ipykernel_101124\2063073377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict['종가'][i] = [float(item) for item in predict['종가'][i].strip('[]').split(',')]


In [74]:
pred_list = predict['종가']
out_labels = predict['종목코드']

In [75]:
# 배열 뺄셈할라면 np.array가 편하긴한데 귀찮아서 그냥 만듬..
def subtract_arrays(arr1, arr2):
    if len(arr1) != len(arr2):
        raise ValueError("Arrays must have the same length")

    result = []
    for i in range(len(arr1)):
        result.append(arr1[i] - arr2[i])
    return result

수익률(benefit) = 예측값 - 마지막 종가
무위험 지표 금리(risk free rate) = Constant한 값
샤프지수 = 수익률 - 지표금리 / 수익률의 표준편차

In [76]:
# 투자 기간과 무위험 수익률 설정
trading_days = 15
risk_free_rate = 0.035

# 주어진 데이터에 따라 샤프 지수 계산
total_return = []
for i in pred_list:
    initial_asset = i[0]  # 초기 자산 (1일의 종가)
    final_asset = i[-1]  # 최종 자산 (15일의 종가)
    total_return.append((final_asset - initial_asset) / initial_asset)

daily_returns = []
for i in pred_list:
    daily_returns.append(i[-1] - i[-2] / i[-2])

annualized_daily_returns = []
for i in pred_list:
    annualized_daily_returns.append([daily_return * 250 for daily_return in daily_returns])

daily_return_volatility = []
for i in range(len(pred_list)):
    daily_return_volatility.append(np.std(annualized_daily_returns[i], ddof=1))

annualized_total_return = []
for i in range(len(pred_list)):
    annualized_total_return.append(total_return[i] * (250 / len(pred_list)))

sharpe_ratio = []
for i in range(len(pred_list)):
    sharpe_ratio.append((annualized_total_return[i] - risk_free_rate) / daily_return_volatility[i])





In [77]:
out_labels

0       A000020
1       A000040
2       A000050
3       A000070
4       A000080
         ...   
1995    A375500
1996    A378850
1997    A383220
1998    A383310
1999    A383800
Name: 종목코드, Length: 2000, dtype: object

In [78]:
df=pd.DataFrame({
        '종목코드': out_labels,
        '샤프지수': sharpe_ratio
    }
)

In [79]:
df = df.sort_values(by='샤프지수', ascending = False)

In [80]:
df.drop(columns=['샤프지수'], inplace = True)

In [81]:
df['순위'] = list(range(1, 2001))

In [57]:
# 이건 종목코드 object로 들어가서 이상하게 나올때만 실행하면 됨
df['종목코드'] = df['종목코드'].astype(str)
df['종목코드'] = df['종목코드'].str.split().str[1]

In [84]:
df.to_csv('{}.csv'.format("230724_v0.5"),index=False)

In [83]:
df

,종목코드,순위
1485,A145210,1
73,A001530,2
77,A001570,3
1469,A140410,4
1549,A183490,5
...,...,...
1493,A149980,1996
1563,A190510,1997
581,A025770,1998
1146,A082800,1999


In [126]:
pred_csv=pd.DataFrame({
        'PredList': pred_list
    }
)

In [127]:
pred_csv.to_csv('{}.csv'.format("pred_list15_230725"),index=False)